In [1]:
from nearline_helpers import *
%jsroot on

Welcome to JupyROOT 6.26/10


In [2]:
infile = '../bin/run00302_00000.mid.root'
f = r.TFile(infile)
t = f.Get("tree")
t.Print()

******************************************************************************
*Tree    :tree      : tree                                                   *
*Entries :     9641 : Total =      1354064261 bytes  File  Size =   85112821 *
*        :          : Tree compression factor =  15.93                       *
******************************************************************************
*Br    0 :waveforms_raw : Int_t waveforms_raw_                               *
*Entries :     9641 : Total  Size=      97562 bytes  File Size  =      59759 *
*Baskets :        6 : Basket Size=      32000 bytes  Compression=   1.30     *
*............................................................................*
*Br    1 :waveforms_raw.fUniqueID : UInt_t fUniqueID[waveforms_raw_]         *
*Entries :     9641 : Total  Size=     124637 bytes  File Size  =     123736 *
*Baskets :        4 : Basket Size=     243712 bytes  Compression=   1.00     *
*...................................................

In [3]:
c = r.TCanvas()
t.Draw("lyso_waveforms.is_clipping","","")
c.Draw()

In [4]:
c = r.TCanvas()
t.Draw("clusters.is_clipping","","")
c.Draw()

In [5]:
# import ROOT as r
# from r import dataProducts

In [6]:
# r.dataProducts.WaveformIntegral()

In [7]:
c = r.TCanvas()
t.Draw("((dataProducts::WaveformIntegral*)(clusters.max_channel@.GetObject())).integral","","")
c.Draw()

In [8]:
c = r.TCanvas()
t.Draw("((dataProducts::WaveformIntegral*)(clusters.max_channel@.GetObject())).integral:clusters.integral","","")
c.Draw()

In [9]:
c = r.TCanvas()
t.Draw("clusters.y:clusters.x","","colz")
c.SetLogz()
c.Draw()

In [ ]:
n = t.Draw("Entry$","clusters.y > 1","goff")
for i in range(n):
    t.GetEntry(int(t.GetV1()[i]))
    break

In [ ]:
[wfi.x for wfi in t.lyso_integrals],[wfi.y for wfi in t.lyso_integrals],[wfi.integral for wfi in t.lyso_integrals],

In [ ]:
for wfi in t.lyso_integrals:
    plot_integral(wfi)
    plt.show()

In [ ]:
def plot_cluster(cluster):
    integrals = [x for x in cluster.xtals]
    fig,ax = plt.subplots(figsize=(8,8))
    ax.set_axis_off()
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    width = 1
    axs = []
    peaks = {}
    for i,inti in enumerate(integrals):
        axs.append(
            ax.inset_axes(
                [inti.x-width/2., inti.y-width/2., width, width],
                transform=ax.transData,
                zorder=(100-inti.x*10)+(100-inti.y*10)
            )
        )
        peaks[i] = inti.peak_time
    for i, axi in enumerate(axs):
        if(i > 0):
            axi.sharex(axs[0])
            axi.sharey(axs[0])
        # else:
        #     axi.set_yticklabels([])
        axi.grid()
        plot_integral(integrals[i], ax=axi, draw_legend=False)
        axi.set_title(f'Channel {integrals[i].channelTag}')
    lims = axi.get_ylim()
    for axi in axs:
        for i, pi in peaks.items():
            axi.plot([pi,pi], lims, "--")
    axi.set_ylim(*lims)
    print(integrals)
    plt.tight_layout()
    
plot_cluster(t.clusters[0])
plt.show()

In [ ]:
limit = 10
for i, e in enumerate(t):
    plot_cluster(e.clusters[0])
    plt.show()
    if(i > limit):
        break